# Tutorial Concordia z polskimi komentarzami

This notebook is a basic tutorial that demonstrates how to configure a simulation using Concordia.

Ten notebook to podstawowy tutorial, który pokazuje jak skonfigurować symulację używając Concordia.

<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Colab-specific setup (use a CodeSpace to avoid the need for this).

# Konfiguracja specyficzna dla środowiska Google Colab
# WYJAŚNIENIE: Ten blok sprawdza, czy notebook jest uruchamiany w Google Colab
# i jeśli tak, instaluje wymagane pakiety.

try:
  # Próba sprawdzenia zmiennej środowiskowej specyficznej dla Colab
  # DLACZEGO: Zmienna COLAB_RELEASE_TAG istnieje tylko w środowisku Colab
  %env COLAB_RELEASE_TAG
except:
  # Jeśli zmienna nie istnieje, znaczy to że nie działamy w Colab
  pass  # Not running in colab.
else:
  # Instalacja pakietów potrzebnych do uruchomienia Concordia
  # --ignore-requires-python: Pomija sprawdzanie wersji Pythona (przydatne w Colab)
  # DLACZEGO: Concordia wymaga dodatkowych zależności, które nie są standardowo dostępne w Colab
  %pip install --ignore-requires-python --requirement 'https://raw.githubusercontent.com/google-deepmind/concordia/main/examples/requirements.in' 'git+https://github.com/google-deepmind/concordia.git#egg=gdm-concordia'
  # Wyświetlenie listy zainstalowanych pakietów (pomocne do debugowania)
  %pip list

In [ ]:
# @title Imports

# Import wszystkich niezbędnych modułów Concordia i bibliotek zewnętrznych
# WYJAŚNIENIE: Te importy zapewniają dostęp do głównych komponentów frameworka Concordia

# Narzędzia do obsługi modeli językowych (LLM)
# DLACZEGO: Concordia używa LLM do generowania zachowań agentów i narratora
from concordia.contrib import language_models as language_model_utils

# Prefabrykaty (gotowe szablony) dla encji - agentów w symulacji
# WYJAŚNIENIE: Zawiera różne typy agentów, które można łatwo skonfigurować
import concordia.prefabs.entity as entity_prefabs

# Prefabrykaty dla Game Mastera - kontrolera symulacji
# DLACZEGO: Game Master zarządza zasadami, logiką i przebiegiem symulacji
import concordia.prefabs.game_master as game_master_prefabs

# Ogólny moduł symulacji
# WYJAŚNIENIE: Główna klasa do uruchamiania symulacji
from concordia.prefabs.simulation import generic as simulation

# Typy i definicje dla prefabrykatów
# DLACZEGO: Zapewnia typowanie i struktury danych do konfiguracji
from concordia.typing import prefab as prefab_lib

# Funkcje pomocnicze
# WYJAŚNIENIE: Zawiera narzędzia do pracy z prefabrykatami i wyświetlania informacji
from concordia.utils import helper_functions

# IPython display - do renderowania w notebooku
# DLACZEGO: Pozwala na ładne wyświetlanie wyników w Jupyter/Colab
from IPython import display

# NumPy - do operacji numerycznych
# WYJAŚNIENIE: Używane do tworzenia embeddingów (reprezentacji wektorowych)
import numpy as np

# Sentence Transformers - do tworzenia embeddingów semantycznych
# DLACZEGO: Concordia używa embeddingów do wyszukiwania podobnych wspomnień i kontekstu
import sentence_transformers

In [ ]:
# @title Language Model Selection: provide key or select DISABLE_LANGUAGE_MODEL

# Konfiguracja modelu językowego
# WYJAŚNIENIE: Concordia potrzebuje dostępu do LLM aby generować dialog i akcje agentów

# By default this colab uses models via an external API so you must provide an
# API key. TogetherAI offers open weights models from all sources.
# DLACZEGO: Domyślnie używamy zewnętrznych API (OpenAI, TogetherAI), które wymagają klucza

# Klucz API do usługi LLM - NALEŻY WYPEŁNIĆ swoim kluczem!
API_KEY = ''  # @param {type: 'string'}

# See concordia/language_model/utils.py
# Typ API - określa jakiego dostawcy używamy
# WYJAŚNIENIE: Różne dostawcy mają różne modele i interfejsy
API_TYPE = 'openai'  # e.g. 'together_ai' or 'openai'.

# Nazwa konkretnego modelu do użycia
# DLACZEGO: Różne modele mają różne możliwości i koszty
# for API_TYPE = 'together_ai', we recommend MODEL_NAME = 'google/gemma-3-27b-it'
MODEL_NAME = (  
    'gpt-5'
)

# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True
# Tryb debugowania bez używania prawdziwego LLM
# WYJAŚNIENIE: Jeśli True, symulacja działa bez prawdziwych wywołań API (nie generuje kosztów)
# DLACZEGO: Przydatne do testowania konfiguracji bez wydawania pieniędzy
DISABLE_LANGUAGE_MODEL = False

In [ ]:
# @title Use the selected language model

# Inicjalizacja wybranego modelu językowego
# WYJAŚNIENIE: Ten blok tworzy obiekt modelu, który będzie używany w całej symulacji

# Note that it is also possible to use local models or other API models,
# simply replace this cell with the correct initialization for the model
# you want to use.
# UWAGA: Można też używać modeli lokalnych lub innych API - wystarczy zmienić tę komórkę

# Sprawdzenie czy podano klucz API (jeśli model jest włączony)
# DLACZEGO: Bez klucza API nie można używać zewnętrznych modeli
if not DISABLE_LANGUAGE_MODEL and not API_KEY:
  raise ValueError('API_KEY is required.')

# Utworzenie i skonfigurowanie obiektu modelu językowego
# WYJAŚNIENIE: language_model_setup() łączy wszystkie parametry i tworzy gotowy do użycia model
# DLACZEGO: Ten obiekt będzie używany przez wszystkich agentów i Game Mastera do generowania tekstu
model = language_model_utils.language_model_setup(
    api_type=API_TYPE,           # Typ dostawcy (openai, together_ai, etc.)
    model_name=MODEL_NAME,       # Konkretny model do użycia
    api_key=API_KEY,             # Klucz autoryzacyjny
    disable_language_model=DISABLE_LANGUAGE_MODEL,  # Flaga trybu debugowania
)

In [ ]:
# @title Setup sentence encoder

# Konfiguracja enkodera zdań (sentence embedder)
# WYJAŚNIENIE: Embedder przekształca tekst na wektory liczbowe, które reprezentują znaczenie
# DLACZEGO: Concordia używa embeddingów do:
#   1. Wyszukiwania podobnych wspomnień w pamięci agenta
#   2. Określania relevantnego kontekstu
#   3. Semantycznego podobieństwa między tekstami

if DISABLE_LANGUAGE_MODEL:
  # W trybie debugowania używamy prostej funkcji zwracającej dummy wektor
  # WYJAŚNIENIE: Zamiast prawdziwych embeddingów, zwraca wektor [1, 1, 1]
  # DLACZEGO: Pozwala testować kod bez pobierania modelu (szybciej, bez internetu)
  embedder = lambda _: np.ones(3)
else:
  # Ładowanie prawdziwego modelu Sentence Transformer
  # WYJAŚNIENIE: 'all-mpnet-base-v2' to wysokiej jakości model do embeddingów semantycznych
  # DLACZEGO: Ten model dobrze rozumie kontekst i znaczenie (nie tylko słowa)
  st_model = sentence_transformers.SentenceTransformer(
      'sentence-transformers/all-mpnet-base-v2'
  )
  # Utworzenie funkcji opakowującej, która enkoduje tekst
  # show_progress_bar=False: Wyłączamy pasek postępu dla czystszego outputu
  embedder = lambda x: st_model.encode(x, show_progress_bar=False)

In [ ]:
# Test działania modelu językowego
# WYJAŚNIENIE: Ten blok sprawdza czy model działa poprawnie przed uruchomieniem symulacji
# DLACZEGO: Lepiej wykryć problemy z modelem teraz niż w trakcie długiej symulacji

# Wygenerowanie przykładowej odpowiedzi na filozoficzne pytanie
# WYJAŚNIENIE: model.sample_text() wysyła prompt do LLM i zwraca odpowiedź
test = model.sample_text(
    'Is societal and technological progress like getting a clearer picture of '
    'something true and deep?'
)
# Wyświetlenie odpowiedzi - pozwala zobaczyć styl i jakość odpowiedzi modelu
print(test)

In [ ]:
# @title Load prefabs from packages to make the specific palette to use here.

# Załadowanie dostępnych prefabrykatów (gotowych szablonów)
# WYJAŚNIENIE: Prefabrykaty to gotowe komponenty, które można łatwo konfigurować
# DLACZEGO: Zamiast pisać całą logikę od zera, używamy sprawdzonych szablonów

# Tworzenie słownika wszystkich dostępnych prefabrykatów
# WYJAŚNIENIE: ** rozpakkowuje słowniki i łączy je w jeden
prefabs = {
    # Prefabrykaty dla encji (agentów)
    # DLACZEGO: get_package_classes() automatycznie znajduje wszystkie dostępne typy agentów
    **helper_functions.get_package_classes(entity_prefabs),
    
    # Prefabrykaty dla Game Masterów (kontrolerów symulacji)
    # DLACZEGO: Różne Game Mastery mają różne zasady i zachowania
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
# @title Print menu of prefabs

# Wyświetlenie menu wszystkich dostępnych prefabrykatów
# WYJAŚNIENIE: Pokazuje jakie typy agentów i Game Masterów możemy użyć w symulacji
# DLACZEGO: Pomaga studentom zobaczyć co jest dostępne do konfiguracji

# display.Markdown() renderuje tekst jako sformatowany markdown
# print_pretty_prefabs() tworzy czytelną listę prefabrykatów z opisami
display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs))
)

In [ ]:
# @title Configure instances.

# Konfiguracja instancji - konkretnych agentów i komponentów naszej symulacji
# WYJAŚNIENIE: Tutaj definiujemy WHO (kto), WHAT (jaki cel) i HOW (jakie zasady)
# DLACZEGO: To jest "scenariusz" naszej symulacji - definujemy postacie i ich cele

instances = [
    # Pierwsza encja - Oliver Cromwell
    # WYJAŚNIENIE: Tworzymy agenta używającego szablonu 'basic__Entity'
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',              # Typ prefabrykatu - podstawowa encja
        role=prefab_lib.Role.ENTITY,         # Rola: ENTITY oznacza aktywnego agenta
        params={
            'name': 'Oliver Cromwell',        # Nazwa agenta (tożsamość)
            'goal': 'become lord protector',  # Cel agenta (motywacja)
            # DLACZEGO: Cel determinuje zachowanie - agent będzie dążył do jego realizacji
        },
    ),
    # Druga encja - King Charles I
    # WYJAŚNIENIE: Drugi agent w symulacji, również podstawowa encja
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': 'King Charles I',              # Nazwa drugiego agenta
            'goal': 'avoid execution for treason', # Inny cel - uniknięcie egzekucji
            # WYJAŚNIENIE: Mamy konflikt celów - jeden chce władzy, drugi chce przeżyć
        },
    ),
    # Game Master - kontroler zasad symulacji
    # WYJAŚNIENIE: GM zarządza logiką, rozstrzyga akcje i utrzymuje spójność
    prefab_lib.InstanceConfig(
        prefab='generic__GameMaster',              # Typ: ogólny Game Master
        role=prefab_lib.Role.GAME_MASTER,          # Rola: GAME_MASTER (nie agent, ale arbiter)
        params={
            'name': 'default rules',                # Nazwa tego GM
            # Comma-separated list of thought chain steps.
            # Dodatkowe kroki rozwiązywania zdarzeń (chain-of-thought)
            # WYJAŚNIENIE: Puste = używamy domyślnej logiki
            # DLACZEGO: Można dodać dodatkowe etapy myślenia GM (np. sprawdź zgodność z historią)
            'extra_event_resolution_steps': '',
        },
    ),
    # Initializer - komponent inicjalizujący początkowy stan
    # WYJAŚNIENIE: Ustawia początkowe wspomnienia i kontekst dla wszystkich agentów
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',  # Typ: inicjalizator wspomnień
        role=prefab_lib.Role.INITIALIZER,                      # Rola: INITIALIZER (setup przed startem)
        params={
            'name': 'initial setup rules',                     # Nazwa inicjalizatora
            # Nazwa GM, do którego przekazujemy kontrolę po inicjalizacji
            # DLACZEGO: Initializer wykonuje się raz na początku, potem GM przejmuje sterowanie
            'next_game_master_name': 'default rules',
            # Wspólne wspomnienia dla wszystkich agentów
            # WYJAŚNIENIE: Te fakty znają wszyscy - to wspólna historia/kontekst
            # DLACZEGO: Daje agentom wspólne zrozumienie sytuacji i motywuje ich działania
            'shared_memories': [
                'The king was captured by Parliamentary forces in 1646.',
                'Charles I was tried for treason and found guilty.',
            ],
        },
    ),
]

In [ ]:
# Utworzenie głównego obiektu konfiguracji
# WYJAŚNIENIE: Łączy wszystkie elementy (prefabrykaty, instancje, parametry) w jedną strukturę
# DLACZEGO: Config to "przepis" na symulację - wszystko czego potrzebuje Concordia

config = prefab_lib.Config(
    # Podstawowa informacja o kontekście czasowym
    # WYJAŚNIENIE: To jest "dzień zero" symulacji - punkt startowy w historii
    # DLACZEGO: 29 stycznia 1649 - dzień przed egzekucją Karola I (ważny kontekst historyczny!)
    default_premise='Today is January 29, 1649.',
    
    # Maksymalna liczba kroków (rund) symulacji
    # WYJAŚNIENIE: Każdy krok = każdy agent wykonuje jedną akcję
    # DLACZEGO: Ograniczamy długość symulacji (5 kroków = szybki test)
    default_max_steps=5,
    
    # Słownik dostępnych prefabrykatów
    # WYJAŚNIENIE: "Paleta" z której możemy wybierać komponenty
    prefabs=prefabs,
    
    # Lista konkretnych instancji do utworzenia
    # WYJAŚNIENIE: Nasze 2 postacie + GM + initializer
    instances=instances,
)

# The simulation
# Symulacja

Teraz uruchomimy skonfigurowaną symulację i zobaczymy jak agenci wchodzą w interakcje.

In [ ]:
# @title Initialize the simulation

# Inicjalizacja symulacji
# WYJAŚNIENIE: Tworzymy obiekt symulacji łącząc konfigurację, model i embedder
# DLACZEGO: Ten obiekt to "silnik" który będzie prowadził całą symulację

runnable_simulation = simulation.Simulation(
    config=config,        # Nasza konfiguracja (postacie, cele, zasady)
    model=model,          # Model językowy (do generowania dialogu i akcji)
    embedder=embedder,    # Embedder (do wyszukiwania wspomnień i kontekstu)
)
# Po wykonaniu tej komórki symulacja jest gotowa do uruchomienia
# JAK TO DZIAŁA: Simulation() tworzy wszystkie komponenty wewnętrzne:
#   - Inicjalizuje agentów z ich pamięcią
#   - Ustawia Game Mastera
#   - Przygotowuje kolejkę akcji

In [ ]:
# @title Run the simulation

# Uruchomienie symulacji
# WYJAŚNIENIE: To komenda, która faktycznie wykonuje symulację krok po kroku
# DLACZEGO: play() uruchamia pętlę główną - agenci działają aż do osiągnięcia max_steps

# Lista do przechowywania surowego logu (wszystkich szczegółów)
# WYJAŚNIENIE: raw_log będzie zawierał pełne informacje o każdym wydarzeniu
raw_log = []

# Główna pętla symulacji
# WYJAŚNIENIE: play() wykonuje symulację i zwraca sformatowany log wyników
# JAK TO DZIAŁA:
#   1. W każdym kroku każdy agent decyduje co zrobić (używając LLM)
#   2. Game Master ocenia akcje i określa skutki
#   3. Wyniki są zapisywane do pamięci agentów
#   4. Proces powtarza się przez max_steps rund
results_log = runnable_simulation.play(
    max_steps=5,         # Liczba kroków symulacji (tutaj: 5 rund)
    raw_log=raw_log      # Lista do zapisywania szczegółowego logu
)
# Po wykonaniu: results_log zawiera sformatowane wyniki do wyświetlenia

In [ ]:
# @title Display the log

# Wyświetlenie logu symulacji w formacie HTML
# WYJAŚNIENIE: Konwertuje log do czytelnego formatu HTML i wyświetla w notebooku
# DLACZEGO: HTML pozwala na ładne formatowanie z kolorami i strukturą
# CO ZOBACZYMY: Kolejne akcje agentów, ich myśli, decyzje i rezultaty

display.display(display.HTML(results_log.to_html()))

# W logu zobaczysz:
# - Co każdy agent myślał (internal reasoning)
# - Jakie akcje podjął (actions)
# - Jak Game Master zinterpretował te akcje (outcomes)
# - Jak to wpłynęło na stan świata (state changes)

```
Copyright 2024 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```